## Autoencoder with single hidden layer

In [1]:
import tensorflow as tf
import numpy as np
from sklearn import datasets

In [2]:
def get_batch(X,size):
    return X[np.random.choice(len(X), size, replace = False)]

class Autoencoder:
    def __init__(self, input_dim, hidden_dim, batch_size = 10, epoch=250, learning_rate=0.001):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.epoch = epoch
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        
        x = tf.placeholder(tf.float32,[None, input_dim])    # N_samples x N_features
        
        with tf.name_scope('encoder'):
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim]), dtype = tf.float32, name='weights') # N_features x N_hidden
            bias = tf.Variable(tf.zeros([1,hidden_dim]), dtype = tf.float32, name = 'bias') # 1 x N_hidden
            encoded = tf.nn.tanh(x @ weights + bias) # N_samples x N_hidden
        with tf.name_scope('decoder'):
            weights = tf.Variable(tf.random_normal([ hidden_dim, input_dim]), dtype = tf.float32, name='weights') # N_hiddenx N_features
            bias = tf.Variable(tf.zeros([1,input_dim]), dtype = tf.float32, name = 'bias')  # 1 x N_features
            decoded = (encoded @ weights + bias) # N_samples x N_features
        
        self.x = x
        self.encoded = encoded
        self.decoded = decoded
            
        self.cost = tf.sqrt(tf.reduce_mean(tf.square(self.x-self.decoded)))
        self.train_op = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.cost)
        # Gradient descent is slowest: Here are other choices -
        # tf.train.AdagradOptimizer, tf.train.AdagradDAOptimizer,
        # tf.train.MomentumOptimizer, tf.train.AdamOptimizer, tf.train.FtrlOptimizer
        self.saver = tf.train.Saver()
        
    def train(self, data):
        num_samples = len(data)
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                for j in range(500):
                    batch_data = get_batch(data, self.batch_size)
                    loss, _ = sess.run([self.cost, self.train_op], feed_dict={self.x : batch_data})
                if i % 10 ==0: 
                    print('epoch {} : loss = {}' .format(i,loss))
                    self.saver.save(sess,'./model.ckpt')
            self.saver.save(sess,'./model.ckpt')
        
    def test(self, data):
        with tf.Session() as sess:
            self.saver.restore(sess,'./model.ckpt')
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict = {self.x : data})
            print('Input : ', data)
            print('Compressed : ', hidden)
            print('Reconstructed : ', reconstructed)
        return reconstructed
                    
                    
                               
        

In [3]:
hidden_dim = 2
data = datasets.load_iris().data

In [4]:
input_dim = len(data[0])
ae = Autoencoder(input_dim, hidden_dim)
ae.train(data)
ae.test([[8, 4, 6, 2]])

Instructions for updating:
Colocations handled automatically by placer.
epoch 0 : loss = 2.808366298675537
epoch 10 : loss = 0.3598375618457794
epoch 20 : loss = 0.29263854026794434
epoch 30 : loss = 0.22453029453754425
epoch 40 : loss = 0.1625186651945114
epoch 50 : loss = 0.24175681173801422
epoch 60 : loss = 0.15716493129730225
epoch 70 : loss = 0.14195756614208221
epoch 80 : loss = 0.21882644295692444
epoch 90 : loss = 0.12144515663385391
epoch 100 : loss = 0.13053126633167267
epoch 110 : loss = 0.16964773833751678
epoch 120 : loss = 0.12869258224964142
epoch 130 : loss = 0.21167388558387756
epoch 140 : loss = 0.1335882693529129
epoch 150 : loss = 0.17544421553611755
epoch 160 : loss = 0.21398349106311798
epoch 170 : loss = 0.19584070146083832
epoch 180 : loss = 0.1731225550174713
epoch 190 : loss = 0.2146349996328354
epoch 200 : loss = 0.18619698286056519
epoch 210 : loss = 0.1584845334291458
epoch 220 : loss = 0.17554248869419098
epoch 230 : loss = 0.1939953714609146
epoch 240 : 

array([[7.7075195, 3.8365688, 5.814247 , 2.0576754]], dtype=float32)